In [74]:
import numpy as np
import pandas as pd 

#import nltk
#nltk.download('stopwords')
#nltk.download('wordnet')
from nltk.corpus import stopwords

from nltk.stem import LancasterStemmer
import re
from nltk.stem import WordNetLemmatizer

from sklearn.utils import shuffle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras import utils
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.preprocessing import text, sequence
from keras.preprocessing.sequence import pad_sequences
from keras.layers import  Dense, Activation, Dropout,Bidirectional, GlobalMaxPool1D,BatchNormalization, Embedding,LSTM, Flatten

Using TensorFlow backend.


In [36]:
sw = stopwords.words('english')
#print(np.array(sw))
stemmer = LancasterStemmer()
lemmatizer = WordNetLemmatizer() 

In [33]:
replace_puncts = {'`': "'", '′': "'", '“':'"', '”': '"', '‘': "'"}

strip_chars = [',', '.', '"', ':', ')', '(', '-', '|', ';', "'", '[', ']', '>', '=', '+', '\\', '•',  '~', '@', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

puncts = ['!', '?', '$', '&', '/', '%', '#', '*','£']

def clean_text(x):
    x = str(x)
    x = x.lower()
    x = re.sub(r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})", "url", x)
    for k, v in replace_puncts.items():
        x = x.replace(k, f' {v} ')
        
    for punct in strip_chars:
        x = x.replace(punct, ' ') 
    
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
        
    x = x.replace(" '", " ")
    x = x.replace("' ", " ")
   
    return x

def stopwords1(text):
    '''a function for removing the stopword'''
    # removing the stop words and lowercasing the selected words
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    # joining the list of words with space separator
    return " ".join(text)


def stemming(text):    
    '''a function which stems each word in the given text'''
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text) 

def lemm(text):    
    text = [lemmatizer.lemmatize(word) for word in text.split()]
    return " ".join(text) 

def text_processing(X):
    X=X.apply(stopwords1)
    X=X.apply(clean_text)
    X=X.apply(stemming)
    X=X.apply(lemm)
    return X

In [63]:
df_fake = pd.read_csv('Fake.csv')
df_true = pd.read_csv('True.csv')
df_fake['is_true'] = 0
df_true['is_true'] = 1

#fake_title = df_fake['title']
true_title = df_true['title']

In [65]:
df_data = pd.concat([df_fake,df_true])

In [69]:
df_data.sample(5)

,title,text,subject,date,is_true
19683,Trump Fan Violently Attacks Comedian For Harm...,Comedian John Caparulo was in the middle of a ...,News,"March 1, 2017",0
4796,"Germany calls for restraint, dialogue in Zimbabwe",BERLIN (Reuters) - Germany called for all side...,worldnews,"November 15, 2017",1
27579,Socialist Utopia of Venezuela Tells People to ...,"GENERALLY, VENEZUELANS DO NOT EAT RABBITS AND...",Government News,"Sep 16, 2017",0
5816,"Irish deputy PM resigns, averting election threat",DUBLIN (Reuters) - Ireland s scandal-hit deput...,worldnews,"November 27, 2017",1
43450,Suicide risk torture victim can be deported: E...,LUXEMBOURG (Reuters) - European governments ca...,worldnews,"October 24, 2017",1


In [68]:
df_data = shuffle(df_data).reset_index(drop=True)

In [70]:
x = df_data['title']
y = df_data['is_true']

In [71]:
clean_x = text_processing(x)

In [75]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state = 42)

In [76]:
y_test_temp = y_test
#max_words = 3000
#tokenize = text.Tokenizer(num_words=max_words)
#tokenize.fit_on_texts(x_train) # only fit on train
#x_train = tokenize.texts_to_sequences(x_train)
#x_test = tokenize.texts_to_sequences(x_test)

encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)
num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [77]:
max_sequences= 300
x_train=pad_sequences(x_train,maxlen=max_sequences)
x_test=pad_sequences(x_test,maxlen=max_sequences)

In [80]:

#LSTM DTS
embed_size = 128

model = Sequential()
model.add(Embedding(max_words,embed_size,input_length=max_sequences))
model.add(Bidirectional((LSTM(64,return_sequences = True,recurrent_dropout=0.5))))
model.add(Dropout(0.6))
model.add(BatchNormalization())
model.add(GlobalMaxPool1D())
model.add(Dense(48, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(len(y.unique()), activation="softmax"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [81]:
batch_size = 64
epochs = 10

#callbacks = [EarlyStopping(monitor='val_loss', patience=4, verbose=1),]
    
history3 =model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
          validation_split=0.2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 25142 samples, validate on 6286 samples
Epoch 1/10
25142/25142 [==============================] - 160s 6ms/step - loss: 0.1820 - accuracy: 0.9236 - val_loss: 0.3816 - val_accuracy: 0.9515
Epoch 2/10
25142/25142 [==============================] - 161s 6ms/step - loss: 0.0901 - accuracy: 0.9677 - val_loss: 0.1447 - val_accuracy: 0.9415
Epoch 3/10
25142/25142 [==============================] - 162s 6ms/step - loss: 0.0649 - accuracy: 0.9755 - val_loss: 0.1580 - val_accuracy: 0.9300
Epoch 4/10
25142/25142 [==============================] - 161s 6ms/step - loss: 0.0517 - accuracy: 0.9797 - val_loss: 0.1578 - val_accuracy: 0.9330
Epoch 5/10
25142/25142 [==============================] - 170s 7ms/step - loss: 0.0414 - accuracy: 0.9840 - val_loss: 0.1197 - val_accuracy: 0.9540
Epoch 6/10
25142/25142 [==============================] - 161s 6ms/step - loss: 0.0352 - accuracy: 0.9862 - val_loss

In [82]:
accr1 = model.evaluate(x_test,y_test)

13470/13470 [==============================] - 27s 2ms/step


In [84]:
print('AKURASI DARI LSTM \nTest set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr1[0],accr1[1]))

AKURASI DARI LSTM 
Test set
  Loss: 0.152
  Accuracy: 0.956
